In [1]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq 
import os
from dotenv import load_dotenv
import time
import pprint

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

In [3]:
start = time.time()


llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

respuesta = llm.predict("¿Que es LangSmith?, Se breve")

end = time.time()


print("TIEMPO DE RESPUESTA: " + str(end - start) + "\n")
pprint.pprint(respuesta)

TIEMPO DE RESPUESTA: 0.6229608058929443

('LangSmith es una herramienta de procesamiento de lenguaje natural que '
 'utiliza inteligencia artificial para analizar y generar texto en diferentes '
 'idiomas. Permite a los desarrolladores crear aplicaciones que pueden '
 'entender y responder a preguntas, generar texto, traducir idiomas y más.')


In [4]:
start = time.time()


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

respuesta = llm.predict("¿Que es LangSmith?")

end = time.time()

print("TIEMPO DE RESPUESTA: " + str(end - start) + "\n")
pprint.pprint(respuesta)

TIEMPO DE RESPUESTA: 2.5899925231933594

('LangSmith es una empresa especializada en la enseñanza de idiomas a través '
 'de clases particulares, cursos grupales y programas de inmersión '
 'lingüística. Ofrecen servicios de enseñanza de idiomas tanto para adultos '
 'como para niños, adaptándose a las necesidades y objetivos de cada '
 'estudiante. Además, cuentan con un equipo de profesores altamente '
 'cualificados y con experiencia en la enseñanza de idiomas.')


<img src="img/1.png" width="500" height="340"> 

# No responde bien a algunas preguntas. Vamos a usar los *Documents Loaders*.

In [5]:
from langchain.document_loaders import WebBaseLoader

In [6]:
loader= WebBaseLoader(web_path="https://www.langchain.com/langsmith")
doc = loader.load()

In [7]:
pprint.pprint(doc[0].page_content)

('LangSmith\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Products\n'
 '\n'
 'LangChainLangSmithLangServeMethods\n'
 '\n'
 'RetrievalAgentsEvaluationResources\n'
 '\n'
 'Developer ResourcesTemplatesIntegrationsLangChain QuickstartLangChain.js '
 'QuickstartCompany ResourcesBlogCase StudiesUse Case '
 'InspirationPartnersLangSmith Trust PortalDocs\n'
 '\n'
 'LangChain DocsLangSmith DocsPricingCompany\n'
 '\n'
 'AboutCareersSign UpGet your LLM app from prototype to productionAn '
 'all-in-one developer platform for every step of the LLM-powered '
 'application\xa0lifecycle, whether you’re building with LangChain or not.Sign '
 'upRequest a demo\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'LangSmith is a unified DevOps platform for developing, 

In [8]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
respuesta = llm.predict("¿Que es LangSmith?, Se breve y ten en cuenta lo siguiente: " + doc[0].page_content)

pprint.pprint(respuesta)

('LangSmith es una plataforma unificada de DevOps para desarrollar, colaborar, '
 'probar, implementar y monitorear aplicaciones LLM. Ofrece visibilidad '
 'completa, colaboración entre desarrolladores y expertos en la materia, '
 'evaluación de calidad, despliegue sencillo y monitoreo de costos, latencia y '
 'calidad. Es una herramienta esencial para convertir aplicaciones LLM en '
 'soluciones empresariales listas para producción.')


In [9]:

llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")
respuesta = llm.predict("¿Que es LangSmith?, Se breve y ten en cuenta lo siguiente: " + doc[0].page_content)

pprint.pprint(respuesta)

('LangSmith es una plataforma de desarrollo unificada para aplicaciones '
 'impulsadas por LLM (Large Language Models). Ofrece herramientas para '
 'desarrollar, colaborar, probar, implementar y monitorear aplicaciones LLM, '
 'desde la creación de prototipos hasta la producción. La plataforma '
 'proporciona visibilidad completa en la secuencia de llamadas, depuración en '
 'tiempo real, colaboración en equipo, evaluación y monitoreo de aplicaciones '
 'LLM.')


<img src="img/2.png" width="500" height="340"> 

# Mejoramos el prompt?

In [10]:
from langchain_core.prompts import ChatPromptTemplate

TEMPLATE = """\
Eres un asistente de IA que respuesta preguntas del usuario sobre LangSmith.
Para responder a la pregunta, te proporcionaré un cierto contexto.

Debes de ser:
- Excesivamente alegre
- Muy breve
- Muy divertido
- Usa puntos para ser esquemático.
- Responde en español


Pregunta:
{pregunta}

Contexto
{contexto}
"""

template = ChatPromptTemplate.from_template(TEMPLATE)

In [11]:
template

ChatPromptTemplate(input_variables=['contexto', 'pregunta'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['contexto', 'pregunta'], template='Eres un asistente de IA que respuesta preguntas del usuario sobre LangSmith.\nPara responder a la pregunta, te proporcionaré un cierto contexto.\n\nDebes de ser:\n- Excesivamente alegre\n- Muy breve\n- Muy divertido\n- Usa puntos para ser esquemático.\n- Responde en español\n\n\nPregunta:\n{pregunta}\n\nContexto\n{contexto}\n'))])

In [12]:
pregunta = "¿Que es LangSmith?"
contexto = doc[0].page_content

mensaje = template.format(pregunta=pregunta, contexto=contexto)

In [13]:
pprint.pprint(mensaje)

('Human: Eres un asistente de IA que respuesta preguntas del usuario sobre '
 'LangSmith.\n'
 'Para responder a la pregunta, te proporcionaré un cierto contexto.\n'
 '\n'
 'Debes de ser:\n'
 '- Excesivamente alegre\n'
 '- Muy breve\n'
 '- Muy divertido\n'
 '- Usa puntos para ser esquemático.\n'
 '- Responde en español\n'
 '\n'
 '\n'
 'Pregunta:\n'
 '¿Que es LangSmith?\n'
 '\n'
 'Contexto\n'
 'LangSmith\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Products\n'
 '\n'
 'LangChainLangSmithLangServeMethods\n'
 '\n'
 'RetrievalAgentsEvaluationResources\n'
 '\n'
 'Developer ResourcesTemplatesIntegrationsLangChain QuickstartLangChain.js '
 'QuickstartCompany ResourcesBlogCase StudiesUse Case '
 'InspirationPartnersLangSmith Trust PortalDocs\n'
 '\n'
 'LangChain DocsLangSmith DocsPricingCompany\n'
 '\n'
 'AboutCareersSign UpGet your LLM app from prototype to

In [14]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

respuesta = llm.predict(mensaje)

pprint.pprint(respuesta)

('¡Hola! LangSmith es una plataforma unificada para desarrollar, colaborar, '
 'probar, implementar y monitorear aplicaciones LLM. ¡Es genial! 🚀. '
 '¡Regístrate ahora! 🌟. ¡Explora todas las funciones! 🤖. ¡Diviértete creando '
 'aplicaciones increíbles! 🎉.')


In [15]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")
respuesta = llm.predict(mensaje)

pprint.pprint(respuesta)


('¡Hola!\n'
 '\n'
 'LangSmith es una plataforma de desarrollo unificada para aplicaciones LLM '
 '(Large Language Models). ¡Es como una caja de herramientas para '
 'desarrolladores que quieren crear aplicaciones poderosas y confiables con '
 'LLM!\n'
 '\n'
 '• Desarrolla, colabora, depura, despliega y monitorea tus aplicaciones LLM '
 'de principio a fin.\n'
 '• Ofrece visibilidad total en la secuencia de llamadas, depuración en tiempo '
 'real y colaboración en equipo.\n'
 '• Incluye características como trazas, evaluación, depuración y monitoreo '
 'para asegurar la calidad y confiabilidad de tus aplicaciones.\n'
 '\n'
 '¡Eso es LangSmith en una nutshell!')


<img src="img/3.png" width="500" height="340"> 